## read data from files

In [205]:
import os
import wget
if os.path.isfile("utilsLaris.py")==True:
    os. remove("utilsLaris.py")
wget.download("https://raw.githubusercontent.com/Ah-essabar/LARIS01/master/utilsLaris.py")

In [1]:
import janitor
import os
import glob
#import mitosheet
import ipywidgets as widgets
%matplotlib inline
#%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import utilsLaris

from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

from sklearn.model_selection import cross_val_score


%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

In [2]:
import pandas as pd

X=pd.read_csv("X_219_02_juin.csv", parse_dates=["date"], index_col="date")
y=pd.read_csv("y_219_02_juin.csv", parse_dates=["date"], index_col="date")

In [3]:
#Timedelta('76 days 23:55:00')
def ts_train_test_split(X,y,test_size=0.35,random_state=42, shuffle=True):
    Xtemp, ytemp = X.copy(), y.copy()
    a= X.index - X.index[0]
    a= a.days
    Xtemp['id']=a
    ytemp['id']=a
    from sklearn.model_selection import train_test_split
    indices_train, indices_test=train_test_split(a.unique(), test_size=test_size,random_state=random_state, shuffle=shuffle)
    X_train = Xtemp[Xtemp['id'].isin(indices_train.values)]
    X_test = Xtemp[Xtemp['id'].isin(indices_test.values)]
    y_train = ytemp[ytemp['id'].isin(indices_train.values)]
    y_test = ytemp[ytemp['id'].isin(indices_test.values)]
    X_train = X_train.drop(["id"], axis=1)
    X_test = X_test.drop(["id"], axis=1)
    y_train = y_train.drop(["id"], axis=1)
    y_test = y_test.drop(["id"], axis=1)
    
    return X_train, X_test, y_train, y_test

In [4]:
X_train, X_test, y_train, y_test = ts_train_test_split(X,y,test_size=0.35,random_state=0, shuffle=True)

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
rfr = RandomForestRegressor()
#y_train=y_train.values.ravel()
rfr.fit(X_train, y_train.values.ravel())
score = rfr.score(X_train, y_train.values.ravel())
print("R-squared:", score) 
y_pred = rfr.predict(X_test)
R_squared = r2_score(y_test.values.ravel(), y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred,squared=False)
print("R-squared: ",R_squared)
print("MSE: ", mse)
print("RMSE: ", rmse) 

R-squared: 0.9904183081833703
R-squared:  0.417610761896852
MSE:  7604.048523514876
RMSE:  87.20119565415875


In [6]:
# sélection des variables à l'aide de randomForest regressor
from sklearn.feature_selection import SelectFromModel
# rfr = clf.fit(xtrain, ytrain)
# clf.feature_importances_  
# array([ 0.04...,  0.05...,  0.4...,  0.4...])
model = SelectFromModel(rfr, prefit=True)
#X_new = model.transform(X_train)
#X_new 

In [7]:
rfrSelected = RandomForestRegressor()
rfrSelected.fit(X_train.loc[:,model.get_support()], y_train.values.ravel())
score_train = rfrSelected.score(X_train.loc[:,model.get_support()], y_train.values.ravel())
score_test = rfrSelected.score(X_test.loc[:,model.get_support()], y_test.values.ravel())
print("R-squared:", score_train, score_test) 

R-squared: 0.9896542019721564 0.3872272887017927


In [8]:
from sklearn.metrics import mean_squared_error
y_pred = rfrSelected.predict(X_test.loc[:,model.get_support()])
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred,squared=False)
print("MSE: ", mse)
print("RMSE: ", rmse) 

MSE:  8000.754694186295
RMSE:  89.44693786925461


In [14]:
X_selected = X.loc[:,model.get_support()].copy()
y_selected = y.copy()

In [21]:
X_selected.to_csv("../HandlingMissingData/X_selected.csv")
y_selected.to_csv("../HandlingMissingData/y_selected.csv")

In [15]:
X_selectedFeatured, y_selectedFeatured =utilsLaris.FeatureExtraction(X_selected,y_selected, period = '30T')

In [16]:
X_trainSF, X_testSF, y_trainF, y_testF = ts_train_test_split(X_selectedFeatured,y_selectedFeatured,test_size=0.35,random_state=0, shuffle=True)

In [17]:
rfrselectedFeatured = RandomForestRegressor()
rfrselectedFeatured.fit(X_trainSF, y_trainF.values.ravel())
score = rfrselectedFeatured.score(X_trainSF, y_trainF.values.ravel())
print("R-squared:", score) 

R-squared: 0.9628990009861725


In [18]:
from sklearn.metrics import mean_squared_error
y_pred = rfrselectedFeatured.predict(X_testSF)
mse = mean_squared_error(y_testF, y_pred)
rmse = mean_squared_error(y_testF, y_pred,squared=False)
print("MSE: ", mse)
print("RMSE: ", rmse) 

MSE:  5449.603130556663
RMSE:  73.82142731319047


In [82]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso

In [83]:
pipeline = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model',Lasso())
])

In [84]:
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(4,7,0.1)},
                      cv = 5, scoring="neg_mean_squared_error",verbose=3
                      )

In [85]:
import warnings
warnings.filterwarnings("ignore")
search.fit(X_train,y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV 1/5] END .............model__alpha=4.0;, score=-13706.055 total time=   0.0s
[CV 2/5] END .............model__alpha=4.0;, score=-13385.864 total time=   0.0s
[CV 3/5] END .............model__alpha=4.0;, score=-21647.292 total time=   0.0s
[CV 4/5] END ...............model__alpha=4.0;, score=-931.655 total time=   0.0s
[CV 5/5] END ..............model__alpha=4.0;, score=-9144.251 total time=   0.0s
[CV 1/5] END .............model__alpha=4.1;, score=-13694.329 total time=   0.0s
[CV 2/5] END .............model__alpha=4.1;, score=-13394.720 total time=   0.0s
[CV 3/5] END .............model__alpha=4.1;, score=-21663.390 total time=   0.0s
[CV 4/5] END ...............model__alpha=4.1;, score=-922.527 total time=   0.0s
[CV 5/5] END ..............model__alpha=4.1;, score=-9089.930 total time=   0.0s
[CV 1/5] END model__alpha=4.199999999999999;, score=-13682.807 total time=   0.0s
[CV 2/5] END model__alpha=4.199999999999999;, 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5,
       6.6, 6.7, 6.8, 6.9])},
             scoring='neg_mean_squared_error', verbose=3)

In [86]:
search.best_params_

{'model__alpha': 5.499999999999995}

In [87]:
coefficients = search.best_estimator_.named_steps['model'].coef_

In [88]:
importance = np.abs(coefficients)

In [89]:
# importance

In [90]:
features = X_train.columns
np.array(features)[importance > 0]

array(['light_100', 'tvoc_103', 'tvoc_105', 'temperature_105',
       'humidity_105', 'co2_108', 'windows'], dtype=object)

In [91]:
# X1 = X['temperature_100_max', 'temperature_105_max', 'co2_111_max',
#        'tvoc_106_std']

In [92]:
selectedFeaturedeatures = ['temperature_100_max', 'temperature_105_max', 'co2_111_max',
       'tvoc_106_std']
selectedFeaturedeatures = np.array(features)[importance > 0]
X_test_selected = X_test[selectedFeaturedeatures].copy()
X_train_selected = X_train[selectedFeaturedeatures].copy()

In [93]:
y_train

,elecWithoutEclairage
date,
2022-03-15 00:00:00,292.932
2022-03-15 00:05:00,302.632
2022-03-15 00:10:00,291.616
2022-03-15 00:15:00,303.436
2022-03-15 00:20:00,294.356
...,...
2022-05-30 23:35:00,308.678
2022-05-30 23:40:00,301.752
2022-05-30 23:45:00,293.822


In [94]:
X_train_selected, y_train= utilsLaris.FeatureExtraction(X_train_selected,y_train, period = '30T')
X_test_selected, y_test=utilsLaris.FeatureExtraction(X_test_selected,y_test, period = '30T')

In [95]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
# RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
#                       max_depth=None, max_features='auto', max_leaf_nodes=None,
#                       maX_selectedamples=None, min_impurity_decrease=0.0,
#                       min_impurity_selectedplit=None, min_samples_leaf=1,
#                       min_samples_split=2, min_weight_fraction_leaf=0.0,
#                       n_estimators=100, n_jobs=None, oob_score=False,
#                       random_state=None, verbose=0, warm_start=False) 

In [96]:
rfr.fit(X_train_selected, y_train)
score = rfr.score(X_train_selected, y_train)
print("R-squared:", score) 

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [90]:
from sklearn.metrics import mean_squared_error
y_pred = rfr.predict(X_test_selected)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred,squared=False)
print("MSE: ", mse)
print("RMSE: ", rmse) 

MSE:  8578.623142363305
RMSE:  92.62085695113873


In [91]:
y_pred.max()

732.0300499999997

In [92]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats

X2 = sm.add_constant(X_train_selected)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

                             OLS Regression Results                             
Dep. Variable:     elecWithoutEclairage   R-squared:                       0.562
Model:                              OLS   Adj. R-squared:                  0.556
Method:                   Least Squares   F-statistic:                     92.23
Date:                  Thu, 02 Jun 2022   Prob (F-statistic):               0.00
Time:                          09:32:51   Log-Likelihood:                -14368.
No. Observations:                  2403   AIC:                         2.880e+04
Df Residuals:                      2369   BIC:                         2.900e+04
Df Model:                            33                                         
Covariance Type:              nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const       

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
estimator = SVR(kernel="linear")
selector = RFE(estimator, n_features_to_select=10, step=1)
selector = selector.fit(X, y)
selector.support_
selector.ranking_

In [ ]:
selector.ranking_

In [ ]:
plt.plot(y_train)
plt.show()

In [ ]:
# from sklearn.datasets import make_friedman1
# from sklearn.feature_selection import RFECV
# from sklearn.svm import SVR
# #X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)
# estimator = SVR(kernel="linear")
# selector = RFECV(estimator, step=1, cv=5)
# selector = selector.fit(X1, y1)
# selector.support_

# selector.ranking_

In [ ]:
# from sklearn.datasets import make_friedman1
# from sklearn.feature_selection import RFE
# from sklearn.svm import SVR
# # X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)
# estimator = SVR(kernel="linear")
# selectorRFE = RFE(estimator, n_features_to_select=10, step=1)
# selectorRFE = selectorRFE.fit(X_train, y_train)
# selectorRFE.support_
# selectorRFE.ranking_

In [ ]:
selectorRFE.feature_names_in_

In [ ]:
colonesSelected = []
for i,j in enumerate(selectorRFE.ranking_):
    # print(j)
    if j<5:
        colonesSelected.append(i)

In [ ]:
var=X_train.columns[colonesSelected]

In [ ]:
var

In [ ]:
var =['temperature_103', 'light_103', 'sound_103', 'temperature_104',
       'light_104', 'light_105', 'light_106', 'temperature_107', 'light_107',
       'tvoc_108', 'sound_108', 'temperature_110', 'temperature_112']

In [ ]:
import pandas
from sklearn import linear_model

X = X[var].copy()
print(X.columns)
col = [0, 4, 5, 7, 8, 9, 13, 14, 15, 17, 18, 21, 25, 26, 29, 30, 32, 33, 34, 35, 36]
col = [0, 4, 5, 7,  9, 13, 15, 18, 21, 26,  30]
col2    =[0, 1, 2, 3, 4, 5,6,  8, 10, 11, 13, 14,15]
# X= X[col]
#X= X.iloc[:,col]
y = y
# importing train_test_split from sklearn
from sklearn.model_selection import train_test_split
# splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, shuffle = False)

X_train = X_train.values
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
datascaler=scaler.fit(X_train)
X_train= datascaler.transform(X_train)
X_test= datascaler.transform(X_test)

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)


print(regr.coef_) 

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats

X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
lm = linear_model.LinearRegression()
X, y=X_train, y_train
lm.fit(X,y)
params = np.append(lm.intercept_,lm.coef_)
predictions = lm.predict(X)

newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(X))
MSE = (sum((y-predictions)**2))/(len(newX)-len(newX.columns))

# Note if you don't want to use a DataFrame replace the two lines above with
# newX = np.append(np.ones((len(X),1)), X, axis=1)
# MSE = (sum((y-predictions)**2))/(len(newX)-len(newX[0]))

var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
sd_b = np.sqrt(var_b)
ts_b = params/sd_b

p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-len(newX.columns)-1))) for i in ts_b]

sd_b = np.round(sd_b,3)
ts_b = np.round(ts_b,3)
p_values = np.round(p_values,3)
params = np.round(params,4)

myDF3 = pd.DataFrame()
myDF3["Coefficients"],myDF3["Standard Errors"],myDF3["t values"],myDF3["Probabilities"] = [params,sd_b,ts_b,p_values]
print(myDF3)

In [ ]:
col= []
for i,pval in enumerate(myDF3["Probabilities"]):
    if pval<=0.001:
        col.append(i-1)
col   = col[1:] 

In [ ]:
col

In [ ]:
# importing r2_score module
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,y_pred)
print("r2 socre is ",score)
print("mean_sqrd_error is ",mean_squared_error(y_test,y_pred))
print("root_mean_squared error of is ",np.sqrt(mean_squared_error(y_test,y_pred)))

In [ ]:
d = {'y_pred': y_pred, 'y_test': y_test}
dfplot= pd.DataFrame(data=d)
dfplot.plot()


## Missing data generation

In [ ]:
#!pip install torch
#!pip install wget 
import wget
#wget.download('https://raw.githubusercontent.com/BorisMuzellec/MissingDataOT/master/utils.py')

import numpy as np
import pandas as pd
from utils import *
import torch
import seaborn as sns

In [ ]:
df1 = df.copy()



In [ ]:
df.plot()

In [ ]:
#!pip install category_encoders
import category_encoders as ce
# Get a new clean dataframe
obj_df = df1.select_dtypes(include=['object']).copy()

In [ ]:
df1 = obj_df.replace(x)

In [ ]:
df1.plot()

In [ ]:
# Fix the seed ------------------------------------------------------
np.random.seed(2)

In [ ]:
# Function produce_NA for generating missing values ------------------------------------------------------

def produce_NA(X, p_miss, mecha="MCAR", opt=None, p_obs=None, q=None):
    """
    Generate missing values for specifics missing-data mechanism and proportion of missing values. 
    
    Parameters
    ----------
    X : torch.DoubleTensor or np.ndarray, shape (n, d)
        Data for which missing values will be simulated.
        If a numpy array is provided, it will be converted to a pytorch tensor.
    p_miss : float
        Proportion of missing values to generate for variables which will have missing values.
    mecha : str, 
            Indicates the missing-data mechanism to be used. "MCAR" by default, "MAR", "MNAR" or "MNARsmask"
    opt: str, 
         For mecha = "MNAR", it indicates how the missing-data mechanism is generated: using a logistic regression ("logistic"), quantile censorship ("quantile") or logistic regression for generating a self-masked MNAR mechanism ("selfmasked").
    p_obs : float
            If mecha = "MAR", or mecha = "MNAR" with opt = "logistic" or "quanti", proportion of variables with *no* missing values that will be used for the logistic masking model.
    q : float
        If mecha = "MNAR" and opt = "quanti", quantile level at which the cuts should occur.
    
    Returns
    ----------
    A dictionnary containing:
    'X_init': the initial data matrix.
    'X_incomp': the data with the generated missing values.
    'mask': a matrix indexing the generated missing values.s
    """
    
    to_torch = torch.is_tensor(X) ## output a pytorch tensor, or a numpy array
    if not to_torch:
        X = X.astype(np.float32)
        X = torch.from_numpy(X)
    
    if mecha == "MAR":
        mask = MAR_mask(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "logistic":
        mask = MNAR_mask_logistic(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "quantile":
        mask = MNAR_mask_quantiles(X, p_miss, q, 1-p_obs).double()
    elif mecha == "MNAR" and opt == "selfmasked":
        mask = MNAR_self_mask_logistic(X, p_miss).double()
    else:
        mask = (torch.rand(X.shape) < p_miss).double()
    
    X_nas = X.clone()
    X_nas[mask.bool()] = np.nan
    
    return {'X_init': X.double(), 'X_incomp': X_nas.double(), 'mask': mask}

In [ ]:
X = df.copy()
shelly_219 =  ['state_AA7BF5','state_AA4706', 'state_AA9130', 'state_483FDA81E0F4']
for shelly in shelly_219:
    X = X.drop([shelly ], axis=1)

In [ ]:
X.columns

In [ ]:
X = X.values
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
datascaler=scaler.fit(X)
X= datascaler.transform(X)

In [ ]:
import pandas as pd
df= pd.DataFrame(X)
df.index=data.index

In [ ]:
df.head()

In [ ]:
#df.to_csv('dataComplet_219.csv')

## MCAR Generation

In [ ]:
# Minimal example for generating missing data ------------------------
p='5.csv'
X_miss_mcar = produce_NA(X, p_miss=0.05, mecha="MCAR")
X_mcar = X_miss_mcar['X_incomp']
R_mcar = X_miss_mcar['mask']

print("Percentage of newly generated missing values: ", (R_mcar.sum()).numpy()/np.prod(R_mcar.size())*100, " %")

df = X_mcar
df = df.numpy()
mask=R_mcar
mask = mask.numpy()

df = pd.DataFrame(data=df, columns=['Temperature', 'Humidity', 'CO2', 'Noise','Pressure'])
mask = pd.DataFrame(data=mask, columns=['Temperature', 'Humidity', 'CO2', 'Noise','Pressure'])

df['time']= data.index
df.set_index('time',inplace = True)
mask['time']= data.index
mask.set_index('time',inplace = True)
df.to_csv('MCAR'+p)
mask.to_csv('mask'+p)

In [ ]:
#df.to_csv('MCAR25.csv')
# supprimer d'abord les deux lignes dans le fichier d'export Neatatmo
index_col="time"
df=pd.read_csv('MCAR40.csv', sep=",", encoding = "ISO-8859-1", index_col=index_col)
mask=pd.read_csv('mask40.csv', sep=",", encoding = "ISO-8859-1", index_col=index_col)
mask=mask.values


In [ ]:
# Sample mar data -----------------------------------------
X_miss_mar = produce_NA(X, p_miss=0.4, mecha="MAR", p_obs=0.4)

X_mar = X_miss_mar['X_incomp']
R_mar = X_miss_mar['mask']

print("Percentage of generated missing values: ", (R_mar.sum()).numpy()/np.prod(R_mar.size())*100, " %")

ax = sns.heatmap(X_mar, mask=R_mar.numpy()==1, linewidths=0.02, linecolor='black')

df = X_mar
df = df.numpy()
mask_mar = R_mar
mask_mar = mask_mar.numpy()

df = pd.DataFrame(data=df, columns=['Temperature', 'Humidity', 'CO2', 'Noise','Pressure'])
mask_mar = pd.DataFrame(data=mask_mar, columns=['Temperature', 'Humidity', 'CO2', 'Noise','Pressure'])



In [ ]:
# df['time']= data.index
# df.set_index('time',inplace = True)
# mask_mar['time'] = data.index
# mask_mar.set_index('time',inplace = True)
# df.to_csv('MAR'+p)
# mask_mar.to_csv('mask_mar'+p)

In [ ]:
# Minimal example for generating missing data ------------------------
p='5.csv'

In [ ]:
pourcentage=5
p=

In [29]:
x= X.iloc[:,0:10]

In [30]:
x

,co2_100,tvoc_100,temperature_100,humidity_100,light_100,sound_100,co2_101,tvoc_101,temperature_101,humidity_101
date,,,,,,,,,,
2022-03-25 16:45:00,431.000000,101.750000,24.277500,18.717500,1023.000000,509.500000,307.000000,51.750000,21.862500,20.172500
2022-03-25 16:50:00,479.000000,119.333333,24.486667,18.933333,1022.000000,510.000000,308.000000,65.000000,22.256667,19.870000
2022-03-25 16:55:00,401.666667,88.000000,23.396667,19.770000,1023.000000,509.666667,355.666667,102.333333,23.266667,19.556667
2022-03-25 17:00:00,435.500000,183.000000,25.515000,17.945000,1022.750000,509.500000,338.750000,197.250000,25.135000,18.025000
2022-03-25 17:05:00,431.333333,230.666667,26.533333,17.183333,1022.666667,509.000000,337.333333,234.333333,26.130000,17.266667
...,...,...,...,...,...,...,...,...,...,...
2022-05-03 23:35:00,412.600000,1079.800000,24.780000,30.770000,316.600000,509.600000,411.200000,1004.800000,24.684000,30.682000
2022-05-03 23:40:00,410.250000,1072.000000,24.770000,30.780000,340.500000,509.250000,410.750000,1002.250000,24.697500,30.697500
2022-05-03 23:45:00,413.750000,1061.000000,24.782500,30.762500,324.750000,509.500000,411.250000,989.250000,24.665000,30.722500


In [10]:
x.shape

(11319, 10)

In [108]:
def idFeature(x,slidingWindows=6):
    # slidingWindowq refer to the nombre of successive obserbations to use for extracte features
    id=[]
    for i,j in enumerate(x.index):
       id.append(i//slidingWindows+1)
    x['id']=id
    x['time']=x.index
    return x

In [35]:
idFeature(x,12)

,co2_100,tvoc_100,temperature_100,humidity_100,light_100,sound_100,co2_101,tvoc_101,temperature_101,humidity_101,id,time
date,,,,,,,,,,,,
2022-03-25 16:45:00,431.000000,101.750000,24.277500,18.717500,1023.000000,509.500000,307.000000,51.750000,21.862500,20.172500,1,2022-03-25 16:45:00
2022-03-25 16:50:00,479.000000,119.333333,24.486667,18.933333,1022.000000,510.000000,308.000000,65.000000,22.256667,19.870000,1,2022-03-25 16:50:00
2022-03-25 16:55:00,401.666667,88.000000,23.396667,19.770000,1023.000000,509.666667,355.666667,102.333333,23.266667,19.556667,1,2022-03-25 16:55:00
2022-03-25 17:00:00,435.500000,183.000000,25.515000,17.945000,1022.750000,509.500000,338.750000,197.250000,25.135000,18.025000,1,2022-03-25 17:00:00
2022-03-25 17:05:00,431.333333,230.666667,26.533333,17.183333,1022.666667,509.000000,337.333333,234.333333,26.130000,17.266667,1,2022-03-25 17:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-03 23:35:00,412.600000,1079.800000,24.780000,30.770000,316.600000,509.600000,411.200000,1004.800000,24.684000,30.682000,943,2022-05-03 23:35:00
2022-05-03 23:40:00,410.250000,1072.000000,24.770000,30.780000,340.500000,509.250000,410.750000,1002.250000,24.697500,30.697500,943,2022-05-03 23:40:00
2022-05-03 23:45:00,413.750000,1061.000000,24.782500,30.762500,324.750000,509.500000,411.250000,989.250000,24.665000,30.722500,944,2022-05-03 23:45:00


In [25]:
from tsfresh import extract_features
features = extract_features(x, column_id="id", column_sort="time")

Feature Extraction: 100%|██████████| 40/40 [02:22<00:00,  3.57s/it]


In [36]:
features.shape

(1887, 7940)

In [42]:
perc = 1 # Like N %
df = features.copy()
min_count =  int(((100-perc)/100)*df.shape[0] + 1)
df = df.dropna( axis=1, 
                thresh=min_count)
print("df.shape :",df.shape)
#msno.matrix(df)

df.shape : (1887, 3005)


In [43]:
df

,light_100__variance_larger_than_standard_deviation,light_100__has_duplicate_max,light_100__has_duplicate_min,light_100__has_duplicate,light_100__sum_values,light_100__abs_energy,light_100__mean_abs_change,light_100__mean_change,light_100__mean_second_derivative_central,light_100__median,...,humidity_100__lempel_ziv_complexity__bins_100,humidity_100__fourier_entropy__bins_2,humidity_100__fourier_entropy__bins_3,humidity_100__fourier_entropy__bins_5,humidity_100__fourier_entropy__bins_10,humidity_100__fourier_entropy__bins_100,humidity_100__permutation_entropy__dimension_3__tau_1,humidity_100__permutation_entropy__dimension_4__tau_1,humidity_100__permutation_entropy__dimension_5__tau_1,humidity_100__permutation_entropy__dimension_6__tau_1
1,0.0,1.0,0.0,1.0,6136.416667,6.275936e+06,0.533333,0.000000,0.166667,1022.875000,...,1.0,0.562335,1.039721,1.039721,1.039721,1.386294,1.039721,1.098612,0.693147,-0.0
2,0.0,1.0,0.0,1.0,6136.500000,6.276106e+06,0.266667,-0.066667,0.000000,1022.666667,...,1.0,0.562335,0.562335,0.562335,0.562335,1.039721,-0.000000,-0.000000,-0.000000,-0.0
3,0.0,0.0,0.0,1.0,6135.250000,6.273549e+06,0.266667,0.033333,0.062500,1022.666667,...,1.0,0.562335,0.562335,0.562335,1.039721,1.386294,-0.000000,-0.000000,-0.000000,-0.0
4,0.0,0.0,1.0,1.0,6132.500000,6.267926e+06,0.100000,-0.100000,0.062500,1022.000000,...,1.0,0.562335,0.562335,1.039721,1.039721,1.386294,-0.000000,-0.000000,-0.000000,-0.0
5,0.0,1.0,0.0,1.0,6131.666667,6.266223e+06,0.066667,-0.066667,-0.041667,1022.000000,...,1.0,0.562335,0.562335,1.039721,1.039721,1.386294,-0.000000,-0.000000,-0.000000,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883,1.0,0.0,0.0,0.0,1858.250000,5.770415e+05,12.630000,-6.450000,6.068750,304.350000,...,1.0,0.562335,0.562335,0.562335,1.039721,1.386294,-0.000000,-0.000000,-0.000000,-0.0
1884,1.0,0.0,0.0,0.0,1897.300000,6.001775e+05,7.040000,2.600000,-0.831250,315.325000,...,1.0,0.562335,0.562335,1.039721,1.386294,1.386294,-0.000000,-0.000000,-0.000000,-0.0
1885,1.0,0.0,0.0,1.0,1946.200000,6.318450e+05,10.090000,2.950000,4.075000,325.525000,...,1.0,0.562335,0.562335,0.562335,1.039721,1.386294,0.562335,0.636514,0.693147,-0.0
1886,1.0,0.0,0.0,0.0,1932.550000,6.229141e+05,8.130000,4.950000,1.931250,317.925000,...,1.0,0.562335,0.562335,0.562335,1.039721,1.386294,-0.000000,-0.000000,-0.000000,-0.0
